In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from scipy.spatial.distance import hamming
import tensorflow
from tensorflow import keras


In [3]:
## function which receives as input number of images and number of labels to get from MNIST datasets
## function returns an array of np arrays in binary form (entries are 1 or 0), ready to be trained
## each entry of array is an np array corresponding to each label, depending on the max_label parameter
## e.g. get_images(10,5) will return an array for labels 0 to 5, with each element containing 10 binarized
## images, where each image is represented by 784 bits (28*28 pixels)

# To display images remove comments below, but BEWARE of the size of the figure and the number of images,
# otherwise it can be VERY slow!
def get_images(num_images,max_label,size):
    images_to_train = []
    #fig, axes = plt.subplots(max_label+1, num_images, figsize=(num_images,2))
    (train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
    for label in range(max_label+1):
        train_images_temp = train_images[(train_labels == label)]
        temp = []
        flattened = []
        binary = []
        for image in range(num_images):
            img = Image.fromarray(train_images_temp[image])
            temp.append(np.array(img.resize((size,size),resample=Image.BILINEAR)))
            #axes[label][image].imshow(temp[image],cmap = "gray")
            #axes[label][image].axis("off")
            flattened.append(temp[image].flatten())
            binary.append(np.where(flattened[image]>0,1,0))
        images_to_train.append(binary)
    #plt.show()

    return images_to_train


In [4]:
## These functions take care of the encoding process of each image. These will essentially map our images into
## the hypervector space

# imported function to generate a memory of HVs, given a seed number, HV dimension, and 
# pixel number (i.e. the image size)
from hdfunctions import gen_im

## This function receives as an input a binary image array and returns an indexHV, i.e. fo every pixel in the
## image, encode it to a hypervector, which is randomly generated with the inner gen_im function, which was 
## imported from the hd functions file.
## The function returns an array of D-dimensional HVs, where each element represents a pixel 
def gen_pixel_hv_array(image_array,seed,D,image_size):
    im = gen_im(seed,D,image_size)
    pixel_hv_array = []
    for image in range(len(image_array)):
        temp = []
        for pixel in range(len(image_array[image])):
            if image_array[image][pixel] == 1:
                temp.append(np.roll(im[pixel],1))
            else:
                temp.append(im[pixel])
        pixel_hv_array.append(temp)
    return pixel_hv_array

# This function takes an array of HV images and bundles the HV pixels together into a single hypervector.
# The bundling works with component-wise addition and majority rule decides whether the value of the sum  
# is 0 or 1, and each image is converted into a single D-dimensional HV
# The function returns an array of the size of the number of images, where each entry is of size D
def bundle_pixel_hv_array(pixel_hv_array):
    bundled_bin_hv_array = []
    for image in range(len(pixel_hv_array)):
        temp = np.sum(pixel_hv_array[image],axis=0)
        bundled_bin_hv_array.append(np.where(temp < len(pixel_hv_array[0])//2,0,1))   
    return bundled_bin_hv_array

# This function receives an array of bundled HV images, and returns a single HV representing the label of the
# encoded images. The output is a list of size D
# This function is also used to encode a test image
def gen_label(bin_hv_array):
    label = []
    sums_array = []
    if len(bin_hv_array) < 2:
        label.append(bin_hv_array)
        return list(label[0][0])
    
    sums_array.append(np.sum(bin_hv_array,axis=0))
    for image in range(len(sums_array)):
        label = np.where(sums_array[image]<=len(bin_hv_array)//2,0,1)
    return label.tolist()

# This function creates the different classes. Receives an array of binary images to be trained and the size
# of the hypervector
# It returns an array of hypervectors where each entry represents the class corresponding 
# to the digit label (associative memory).
def gen_am(images_to_train,D,size):
    AM = []
    for array in range(len(images_to_train)):
        pixel_hv_array = gen_pixel_hv_array(images_to_train[array],0,D,size*size)
        bundled_hv_array = bundle_pixel_hv_array(pixel_hv_array)
        label = gen_label(bundled_hv_array)
        AM.append(label)
    return AM

In [5]:
num_images = 3000
max_label = 9
size = 15

In [6]:
get_images_test = get_images(2000,9,15)

In [7]:
new_am_test = gen_am(get_images_test,10000,15)

In [8]:
# This function returns a specified number of query images to test our HV implementation 

def test_images_function(num_images,size):
        (test_images, test_labels) = keras.datasets.mnist.load_data()[1]
        test_images_array, labels = test_images[:num_images],test_labels[:num_images]
        binary_images = []
        for label in range(len(labels)):
                image = Image.fromarray(test_images_array[label]).resize((size,size),resample=Image.BILINEAR)
                im_arr = np.array(image)
                flat_im = im_arr.flatten()
                binary_im = [np.array((np.where(flat_im>0,1,0)))]
                binary_images.append(gen_label(bundle_pixel_hv_array(gen_pixel_hv_array(binary_im,0,10000,size*size))))
        return binary_images, labels


In [9]:
binary_images, labels = test_images_function(50,15)


In [10]:
tested_values = []
for i in range(len(binary_images)):
    hammings = []
    for j in range(len(new_am_test)):
        hammings.append(hamming(binary_images[i],new_am_test[j]))
    min_value = hammings.index(min(hammings))
    tested_values.append(min_value)
print(list(labels))
print(tested_values)

result = []
for i in range(len(labels)):
    result.append(tested_values[i] - labels[i])
print(result)

correct_labels = []
for i in range(len(result)):
    if result[i] == 0:
        correct_labels.append(result[i])
print(f"Number of correct images: {len(correct_labels)}")
print(f"Accuracy: {len(correct_labels)/len(result)}")


[7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4]
[7, 2, 1, 0, 4, 1, 4, 9, 5, 7, 0, 0, 9, 0, 1, 3, 9, 7, 2, 4, 9, 6, 4, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 0, 7, 2, 7, 1, 3, 1, 1, 7, 4, 2, 3, 8, 9, 2, 4, 4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, -2, 0, -6, 0, 0, 0, -2, 0, 0, -1, 0, 0, 0, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 8, 0, 0, 0]
Number of correct images: 41
Accuracy: 0.82
